# Homework #4

In [1]:
%%capture
!pip install sentence_transformers

In [11]:
import pandas as pd
from tqdm.auto import tqdm

In [3]:
github_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv"

In [4]:
url = f'{github_url}?raw=1'
df = pd.read_csv(url)

## Q1 Getting the embeddings model

In [5]:
model_name = "multi-qa-mpnet-base-dot-v1"

In [6]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
answer_llm = df.iloc[0].answer_llm
answer_llm

'You can sign up for the course by visiting the course page at [http://mlzoomcamp.com/](http://mlzoomcamp.com/).'

In [8]:
embedding_llm = embedding_model.encode(answer_llm)
embedding_llm[0]

-0.42244658

## Q2 Computing the dot product

In [9]:
results_gpt4omini = df.to_dict(orient='records')

In [10]:
def compute_similarity(record):
    answer_orig = record['answer_orig']
    answer_llm = record['answer_llm']

    v_llm = embedding_model.encode(answer_llm)
    v_orig = embedding_model.encode(answer_orig)

    return v_llm.dot(v_orig)

In [12]:
evaluations = []

In [13]:
for record in tqdm(results_gpt4omini):
    sim = compute_similarity(record)
    evaluations.append(sim)

  0%|          | 0/1830 [00:00<?, ?it/s]

In [15]:
df['evaluations'] = evaluations
df['evaluations'].describe()

,evaluations
count,1830.000000
mean,28.015772
std,6.413295
min,3.511811
25%,24.631171
50%,28.897569
75%,32.389797
max,44.296780


In [20]:
df.iloc[25]

,25
answer_llm,If you haven't received a confirmation email a...
answer_orig,"The process is automated now, so you should re..."
document,6ba259b1
question,What should I do if I haven't received a confi...
course,machine-learning-zoomcamp
cosine,37.980003
evaluations,37.980003


In [21]:
compute_similarity(df.iloc[25])

37.980003

# Q3 Computing the cosine

In [32]:
def _normalize(v):
    norm = np.sqrt(np.sum(np.square(np.array(v))))
    return v / norm

In [33]:
def compute_similarity_normalized(record):
    answer_orig = record['answer_orig']
    answer_llm = record['answer_llm']

    v_llm = embedding_model.encode(answer_llm)
    v_orig = embedding_model.encode(answer_orig)

    v_llm_norm = _normalize(v_llm)
    v_orig_norm = _normalize(v_orig)

    return v_llm_norm.dot(v_orig_norm)

Test it

In [34]:
compute_similarity_normalized(df.iloc[25])

0.92667866

Run it

In [36]:
evaluations_norm = []

In [37]:
for record in tqdm(results_gpt4omini):
    sim = compute_similarity_normalized(record)
    evaluations_norm.append(sim)

  0%|          | 0/1830 [00:00<?, ?it/s]

In [38]:
df['evaluations_norm'] = evaluations_norm
df['evaluations_norm'].describe()

,evaluations_norm
count,1830.000000
mean,0.738963
std,0.156405
min,0.090286
25%,0.655194
50%,0.772145
75%,0.853112
max,0.993684


## Q4 Rouge

In [39]:
%%capture
!pip install rouge

In [43]:
r = df.iloc[10].to_dict()

In [44]:
from rouge import Rouge
rouge_scorer = Rouge()

scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]

In [45]:
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

## Q5 Average rouge score

In [49]:
f_scores = [scores['rouge-l']['f'], scores['rouge-1']['f'], scores['rouge-2']['f']]
np.average(f_scores)

0.35490034990035496

## Q6 Average rouge score for all the data points

In [50]:
def compute_rouge(record):
    answer_orig = record['answer_orig']
    answer_llm = record['answer_llm']

    scores = rouge_scorer.get_scores(record['answer_llm'], record['answer_orig'])[0]
    # f_scores = [scores['rouge-l']['f'], scores['rouge-1']['f'], scores['rouge-2']['f']]

    # return np.average(f_scores)

In [52]:
score_rouge_2 = []

In [53]:
for record in tqdm(results_gpt4omini):
    scores = compute_rouge(record)
    score_rouge_2.append(scores['rouge-2']['f'])

  0%|          | 0/1830 [00:00<?, ?it/s]

In [54]:
np.average(score_rouge_2)

0.17671704698262222